In [163]:
import paramiko
import pandas as pd
import time

In [164]:

switch_groups = pd.read_csv('switch_groups.txt', sep='\t')
switch_groups["login"] = None
switch_groups["password"] = None
switch_groups

,group,hint,login,password
0,1,switch ze scianki,None,None
1,2,switch ze scianki,None,None
2,3,switch ze scianki,None,None
3,4,switch ze scianki,None,None


In [165]:
switch_ips = pd.read_csv('correct_ips.txt', sep='\t')
switch_ips

,group,swt,ip
0,1,SWITCH2,192.168.1.54
1,2,RandomSwitch,192.168.1.55
2,3,JustTest,192.168.1.56
3,4,NotOK,192.168.1.57


In [166]:
for index, row in switch_groups.iterrows():
    group = row['group']
    hint = row['hint']
    password = None
    login = None

    while not login:
        login = input(f"Wprowadz login do switchy z grupy {group} (podpowiedz: ({hint})): ")
        if not login:
            print("Login pusty, proszę wprowadz ponownie")

    while not password:
        password = input(f"Wprowadz haslo do switchy z grupy {group} (podpowiedz: ({hint})): ")
        if not password:
            print("Haslo puste, prosze wprowadz ponownie")
    
    switch_groups.at[index, 'login'] = login
    switch_groups.at[index, 'password'] = password

switch_groups

,group,hint,login,password
0,1,switch ze scianki,admin,switch
1,2,switch ze scianki,a,a
2,3,switch ze scianki,a,a
3,4,switch ze scianki,a,a


In [167]:
merged_df = pd.merge(switch_ips, switch_groups[['group', 'login', 'password']], on='group', how='left')
merged_df

,group,swt,ip,login,password
0,1,SWITCH2,192.168.1.54,admin,switch
1,2,RandomSwitch,192.168.1.55,a,a
2,3,JustTest,192.168.1.56,a,a
3,4,NotOK,192.168.1.57,a,a


In [168]:
try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(merged_df['ip'][0], username=merged_df["login"][0], password=merged_df["password"][0])
except:
    print("brak połączenia z")

In [169]:
stdin, stdout, stderr = ssh.exec_command('sh mac address-table | exclude (Po|Vl|CPU)')

data = stdout.read().decode('utf-8')
print(data)

          Mac Address Table
-------------------------------------------

----    -----------       --------    -----
   1    f8e4.3bcc.75eb    DYNAMIC     Gi1/1
Total Mac Addresses for this criterion: 27



In [170]:
# Split the string into lines
lines = data.strip().split('\n')

# Remove header and separators
lines = [line for line in lines if not line.startswith(('Mac Address Table', '-------------------------------------------', 'Vlan    Mac Address       Type        Ports', '----    -----------       --------    -----'))]

# Define columns
columns = ['vlan', 'mac address', 'type', 'ports']

df = pd.DataFrame(columns=columns)

# Parse each line and extract the columns
for line in lines:
    words = line.strip().split()
    # print(words)
    if len(words) > 0 and words[0] != 'Total': #this is condition to get rid of first empty line and last summary line
        row = {'vlan': words[0], 'mac address': words[1], 'type': words[2], 'ports': words[3]}
        df.loc[len(df)] = row
        

df

,vlan,mac address,type,ports
0,1,f8e4.3bcc.75eb,DYNAMIC,Gi1/1


In [171]:
ssh.close()